In [1]:
## packages installed

import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
import tensorflow as tf
import seaborn
import keras
import mediapipe as mp 
import pathlib
import cv2
import string

2023-11-04 15:19:19.520033: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Load Data 

In [2]:
## TO-DO: Load the ASL Alphabet dataset, using keras.preprocessing.image 

from keras.preprocessing.image import ImageDataGenerator
import os
# train_data= pathlib.Path(os.path.expanduser("~/Desktop/DAML_ASL/archive/asl_alphabet_train/asl_alphabet_train")).with_suffix('')
# train_ds = tf.keras.utils.image_dataset_from_directory(
#   train_data,
#   validation_split=0.2,
#   subset="training",
#   seed=123)

# val_ds = tf.keras.utils.image_dataset_from_directory(train_data,
#   validation_split=0.2,
#   subset="validation",
#   seed=123)
data_dir=pathlib.Path(os.path.expanduser("~/Desktop/DAML_ASL/archive")).with_suffix('')





In [3]:
""" train_ds.class_names
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(train_ds.class_names[labels[i]])
    plt.axis("off") """

' train_ds.class_names\nimport matplotlib.pyplot as plt\nplt.figure(figsize=(10, 10))\nfor images, labels in train_ds.take(1):\n  for i in range(9):\n    ax = plt.subplot(3, 3, i + 1)\n    plt.imshow(images[i].numpy().astype("uint8"))\n    plt.title(train_ds.class_names[labels[i]])\n    plt.axis("off") '

### MediaPipe

##### check https://developers.google.com/mediapipe/solutions/vision/hand_landmarker/python for guidance on this section

In [4]:
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

In [5]:
## TO-DO: Download and Load Model Path

model_path = os.path.expanduser('~/Desktop/DAML_ASL/daml-aslalphabet/hand_landmarker.task')


In [6]:
## TO-DO: prepare image data, create task, and run the task

import mediapipe as mp

BaseOptions = mp.tasks.BaseOptions
HandLandmarker = mp.tasks.vision.HandLandmarker
HandLandmarkerOptions = mp.tasks.vision.HandLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode
print(model_path)
options = HandLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    running_mode=VisionRunningMode.IMAGE)

        
        



/Users/jackkugler/Desktop/DAML_ASL/daml-aslalphabet/hand_landmarker.task


In [14]:

def process_output(landmarks, Y):
    removal =[]
    Y_new = Y
    return_output = []
    print(len(landmarks))
    count = 0
    for i in range(len(landmarks)):
        return_data = []
        if len(landmarks[i].hand_landmarks) == 0:
            removal.append(i)
            continue
        for NormaLandmark in landmarks[i].hand_landmarks[0]:
            return_data.append(NormaLandmark.x)
            return_data.append(NormaLandmark.y)
        return_output.append(return_data)
    for x in reversed(removal):
       Y_new.pop(x)
    return return_output, Y_new



    """
    extract and clean landmark coordinate data: including 
    - Removing depth (or z dimension)
    - Centralizing coordinates to center point of hand
    - Flattening to 1D array 
    - Normalization w.r.t. max value 
 
    Paramaters
    ---------
    landmarks: output from Mediapipe model 
    """
Y_train = []
X_train = []
Y_test = []
X_test = []
alphabet = list(string.ascii_uppercase)
with HandLandmarker.create_from_options(options) as landmarker:
    for letter in alphabet:
        count = 0 
        #print(letter)
        for images in data_dir.glob(f'*/asl_alphabet_train/{letter}/*.jpg'):
            #letter = str(images).split('asl_alphabet_train/asl_alphabet_train/',1)[1][0]
            Y_train.append(letter)
            mp_image = mp.Image.create_from_file(str(images))
            hand_landmarker_result = landmarker.detect(mp_image)
            X_train.append(hand_landmarker_result)
            count+=1
            if count>10:
                break
# count = 0 
# with HandLandmarker.create_from_options(options) as landmarker:
#     for images in data_dir.glob('*/asl_alphabet_test/*/*.jpg'):
#         letter = str(images).split('asl_alphabet_test/asl_alphabet_test/',1)[1][0]
#         Y_test.append(letter)
#         #print("Here")
#         mp_image = mp.Image.create_from_file(str(images))
#         mp_image = mp.Image.create_from_file(str(images))
#         hand_landmarker_result = landmarker.detect(mp_image)
#         X_test.append(hand_landmarker_result)
#         count +=1
#         if count > 100:
#             break

markers, letters = (process_output(X_train,Y_train))
print(len(markers))
print(len(letters))
X_trainMP, X_testMP, Y_trainMP, Y_testMP = train_test_split(markers, letters,test_size=0.2, random_state=123)
print(len(X_trainMP))
print(len(Y_trainMP))
# X_test, Y_test = process_output(X_test, Y_test)
#data_tot

286
235
235
188
188


In [47]:

##TO-DO: generate the train and test set
#X_train, y_train, X_test, y_test =  sklearn.model_selection.train_test_split()

TypeError: cannot unpack non-iterable ellipsis object

### Model Training with ResNet / VGG

In [9]:
##TO-DO: train and evaluate model

### Model Training with Mediapipe Landmarks (Random Forest, LSTM, etc.)

In [17]:
##TO-DO: train and evaluate model
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators = 500, max_depth = 4, max_features = 3, bootstrap = True, random_state = 18).fit(X_trainMP, Y_trainMP)
# Create our predictions
prediction = rf.predict(X_testMP)
# Create confusion matrix
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
confusion_matrix(Y_testMP, prediction)
# Display accuracy score
#accuracy_score(Y_testMP, prediction)
# Display F1 score
#f1_score(Y_testMP,prediction)



0.3404255319148936